In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

import brighteyes_ism.simulation.PSF_sim as sim
import brighteyes_ism.analysis.Graph_lib as gra
import brighteyes_ism.dataio.mcs as mcs

from s2ism import s2ism as s2
import s2ism.psf_estimator as est

import brighteyes_flim.flism as flim

# Data download

Experimental dataset can be downloaded from [https://doi.org/10.5281/zenodo.11284051](https://doi.org/10.5281/zenodo.11284051).

Here, we download and decompress the data into the Downloas folder.

In [2]:
import requests
from tqdm import tqdm, trange
from pathlib import Path
import os
import zipfile

def download(url, fname):
    resp = requests.get(url, stream=True)
    total = int(resp.headers.get('content-length', 0))
    with open(fname, 'wb') as file, tqdm(
            desc=fname,
            total=total,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
    ) as bar:
        for data in resp.iter_content(chunk_size=1024):
            size = file.write(data)
            bar.update(size)
            
def extract(filename):
    dir_name = filename[:-4]
    os.mkdir(dir_name)
    with zipfile.ZipFile(filename, 'r') as zf:
        for member in tqdm(zf.infolist(), desc='Extracting '):
            try:
                zf.extract(member, dir_name)
            except zipfile.error as e:
                pass

In [3]:
downloads_path = str(Path.home() / 'Downloads')

url_data = 'https://zenodo.org/records/11284051/files/h5_files_s2ism.zip'
name_data = 'h5_files_s2ism.zip'

filename = os.path.join(downloads_path, name_data)

In [4]:
if not os.path.isfile(filename):
    print('Downloading data:' + filename + '\n')
    download(url_data, filename)
else:
    print('File already downloaded.')

File already downloaded.


In [7]:
dir_name = filename[:-4]

if not os.path.exists(dir_name):
    print('Extracting compressed data:\n')
    extract(filename)

# Data loading

In [9]:
file = r'Lamina_tubulin_flim_data-10-04-2024-19-27-47.h5'

fullpath = os.path.join(dir_name, file)

data, meta = mcs.load(fullpath)

print('Loaded data: ' + file)

dset = np.squeeze(data)

del data

data_extra, _ = mcs.load(fullpath, key="data_channels_extra")
laser = data_extra[:, :, :, :, :, 1].sum((0, 1, 2, 3)) * 1.
laser /= laser.max()

Loaded data: Lamina_tubulin_flim_data-10-04-2024-19-27-47.h5


In [ ]:
exPar = sim.simSettings()
exPar.na = 1.4   # numerical aperture
exPar.wl = 488   # wavelength [nm]
exPar.gamma = 45  # parameter describing the light polarization
exPar.beta = 90  # parameter describing the light polarization
exPar.n = 1.5
exPar.mask_sampl = 100

emPar = exPar.copy()
emPar.wl = 520

grid = sim.GridParameters()
grid.Nz = 2
grid.pxsizex = meta.dx*1e3
grid.pxsizez = 720
grid.pxpitch = 75e3
grid.pxdim = 50e3
grid.N = 5

psf_spatial, _,_ = est.psf_estimator_from_data(dset.sum(-2), exPar, emPar, grid, z_out_of_focus = 300)


In [ ]:
file_irf = 'IRF_for_lamina_tubulin_flim_data-10-04-2024-19-50-34.h5'

print(f'Loading: {file}')
data, meta = mcs.load(file)


In [ ]:
dset_irf = np.sum( data, axis = (0, 1, 2, 3) ) * 1.
dset_irf /= dset_irf.max(-2)

In [ ]:
data_extra_irf, _ = mcs.load(file, key="data_channels_extra")
laser_irf = data_extra_irf[:, :, :, :, :, 1].sum((0, 1, 2, 3))
laser_irf /= laser_irf.max()

plt.figure()
plt.plot(data_laser_irf)

In [ ]:
#dt = 0.297619 # dfd bin time, ns
dt = data_extra_irf['dt'] # ns
time = np.arange(dset.shape[0]) * dt

In [ ]:
plt.figure()

for n in range(dset.shape[-1]):
    plt.plot(time, dset[:, n] + n)

plt.xlabel('time (ns)')

In [ ]:
def centroid(data):
    x = np.arange(data.size)
    idx = np.sum(x*data)/data.sum()
    return idx

def clean_irf(irf, threshold = 0.3, window = 6):
    
    time = np.arange(irf.size)
    
    t_irf = np.where(irf>threshold, irf, 0)
    
    t0 = centroid(t_irf)
    
    indices = np.argwhere(np.logical_and(time > t0 - window, time < t0 + window))
    
    final_irf = np.zeros_like(irf)
    
    final_irf[indices] = irf[indices]
    
    return final_irf

In [ ]:
final_irf = np.empty_like(dset_irf)

for n in range(dset_irf.shape[-1]):

    irf = dset_irf[:, n]
 
    final_irf[:, n] = clean_irf(irf, threshold = 0.3, window = 2/dt)
    
    plt.figure()
    plt.plot(time, irf)
    plt.plot(time, final_irf[:, n])

In [ ]:
psf_irf = est.combine_psf_irf(psf_spatial, final_irf)

In [ ]:
s2_rec = s2.batch_reconstruction(dset, psf_irf, batch_size = [301, 301], overlap = 40, max_iter = 5, process='gpu')
